# 16b. Green band classifier tool
The color analysis outputs the percent of pixels at a given depth which are green. The question is: What meaning does this percent green have? Is it actually meaningful on its own? The current plan is to set a threshold for a percent green-ness that will constitute a binary positive confirmation of a green layer. 

## Setup
### Import Modules

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator
# %matplotlib inline
from scipy import signal,interpolate,stats,linalg
# from IPython.display import clear_output
import time
import numpy as np
import seaborn as sns
from PIL import Image
import os
import cv2 as cv
import pickle
import random

### Set paths

In [2]:
data_path='/Users/danielbabin/GitHub/Green_Bands/Data/'
table_path='/Users/danielbabin/GitHub/Green_Bands/Tables/'
saturated_cores='/Volumes/SanDisk.Data/IODP1313CorePhotos/Saturated/'
filtered_cores='/Volumes/SanDisk.Data/IODP1313CorePhotos/Filtered/'
raw_cores='/Volumes/SanDisk.Data/IODP1313CorePhotos/Cropped/'
figure_path='/Users/danielbabin/GitHub/Green_Bands/Figures/Main/'
sup_figure_path='/Users/danielbabin/GitHub/Green_Bands/Figures/Supplementary/'
checkpoints_path='/Users/danielbabin/GitHub/Green_Bands/Data/Checkpoints/'

### Import Splice info

In [3]:
splice=pd.read_csv(data_path+'SpliceTables/cleanSpliceConstruction1313.csv')
splice['CoreLabel']=(splice['Site'].map(str).str.strip()+splice['Hole'].str.strip()+'_'+splice['Core'].map(str))

section_list=pd.read_csv(data_path+'SectionSummaries/spliceSections1313.csv')
section_list['SectionLabel']=(section_list['Site'].map(str)+section_list['Hole']+'_'+
                              section_list['Core'].map(str)+section_list['Type']+
                              section_list['Section'].map(str))
section_list=section_list.set_index('SectionLabel',drop=False)

In [13]:
sections=section_list[(section_list['Top(mcd)']<56)&
                      (section_list['Section']!='CC')]['SectionLabel'].reset_index(drop=True)
sections=section_list.loc[sections]

### Import Green

In [14]:
green=pd.read_csv(table_path+'Supplementary Table 5A - U1474 green intensity splice.csv',
                  index_col='CCSF Depth (m)')
green['SectionLabel']=(green['Site'].map(str)+green['Hole']+'_'+
                       green['Core'].map(str)+green['Type']+
                       green['Section'].map(str))
green=green[green.index.duplicated()==False]
green=green.sort_index()
green_channel_cols=green.columns[7:-1]
green.loc[green['Green%']<=1,'Green%']=0

## Core Photos

In [15]:
greens=pd.read_csv(checkpoints_path+'green.peak.labeling.U1313.csv',index_col='Top Depth CCSF (m)')

### Define Threshold-Picking Function

In [16]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [17]:
def check(n):
    greens=pd.read_csv(checkpoints_path+'green.peak.labeling.U1313.csv',index_col='Top Depth CCSF (m)')
    sample=greens.iloc[n]
    top_depth=sample.name
    bottom_depth=sample['Bottom Depth CCSF (m)']
    top_offset=sample['Top Offset (cm)']/100
    bottom_offset=sample['Bottom Offset (cm)']/100
    sect=sample['SectionLabel']
    hole=sect[4]
    number=1
    buffer=.2
    greendata=green[green['SectionLabel']==sect]
    greendata=greendata.loc[top_depth-buffer:bottom_depth+buffer]

    fig=plt.figure(figsize=(14,7))
    gs = gridspec.GridSpec(8, 1, figure=fig,hspace=.02,left=.05,right=.95,top=.95)

    c1=fig.add_subplot(gs[:3,:])
    c2=fig.add_subplot(gs[3:6,:])
    ax1=fig.add_subplot(gs[6:8,:])

    c1.set_title('Hole '+sect[4]+' Core '+sect[6:],
                  fontweight='bold',fontsize=20,pad=0,loc='left')
    c1.annotate('Number '+str(number+1)+'/213',xy=(1,1),xycoords='axes fraction',
                fontweight='bold',fontsize=20,horizontalalignment='right',verticalalignment='bottom')
    
    sect_length=(section_list.loc[sect,'Bottom(mcd)']-
                  section_list.loc[sect,'Top(mcd)'])
    
    raw_img=Image.open(raw_cores+sections.loc[sect,'Filename'])
    filt_img=Image.open(filtered_cores+sections.loc[sect,'Filename'])
    
    h=raw_img.size[1]
    w=raw_img.size[0]
    aspect=(w/h)
    naspest=((16.6/aspect)-1)/2
    
    
    pxl_m=w/sect_length
    extentlist=[top_depth-buffer,
                bottom_depth+buffer,
                0,sect_length/aspect]
    sample_top_pxl=int(top_offset*pxl_m)
    sample_bottom_pxl=int(bottom_offset*pxl_m)
    pxl_offset=int(buffer*pxl_m)
    
    raw=raw_img.crop((sample_top_pxl-pxl_offset,0,sample_bottom_pxl+pxl_offset,h))
    filt=filt_img.crop((sample_top_pxl-pxl_offset,0, sample_bottom_pxl+pxl_offset,h))
    
    c1.imshow(raw,extent=extentlist)
    c1.axis('off')
    
    c2.imshow(filt,extent=extentlist)
    c2.axis('off')
    
#     data.plot(ax=ax1,marker='o',ms=3,c=colors[-3])
    greendata['Green%'].plot(kind='area',ax=ax1,color='tab:green')
#     ax1.set_ylim(0,100)
    
    cols=['% Pixels']
    for i,a in enumerate([ax1]):
        a.set_ylabel(cols[i])
    for i,a in enumerate([c1,c2,ax1]):
        a.set_xlim(top_depth-buffer,bottom_depth+buffer)

    ## Annotate
    ax1.set_xlabel('Depth (m CCSF)');
    box=dict(boxstyle="round",ec=(0,0,0),fc=(1,1,1),clip_on=True)
    for ax in [ax1]:
        ax.axvline(top_depth,color='k')
        ax.axvline(bottom_depth,color='k')
        ax1.annotate(n,xy=(bottom_depth+.015,greendata['Green%'].max()*.9),ha='left',size=16,bbox=box)
    
    ## Question
    print('Is ',n,'\n',
          '(1) a diffuse, mottled green layer?\t',
          '(2) a diffuse, continuous green layer?\n',
          '(3) a crusty, mottled green layer?\t',
          '(4) a crusty, continuous green layer?\n',
          '(5) flag algorithm problem\t\t',
          '(0) not a green layer')
    plt.show(block=False)
    gTF=int(input())
    plt.close()
    greens.loc[top_depth,'Label']=gTF
    greens.to_csv(checkpoints_path+'green.peak.labeling.U1313.csv')

### Check

In [19]:
print('Do you need to classify green bands?: (y/n)')
classify=str(input())
if classify=='y':
    print('Enter start n: ')
    start_n=int(input())
    for i in range(start_n,len(greens)):
        check(i)

In [ ]:
greens_labelled=pd.read_csv(table_path+'green.peak.labeling.U1313.csv')
# greens_labelled.to_csv(table_path+'Supplementary Table 6B - U1313 green bands.csv',index=False)